<a href="https://colab.research.google.com/github/ZiChengHao/Mat-494/blob/main/homework7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

4.2 Spectral Clustering

In [ ]:
# import libraries
import numpy as np
import scipy
import pandas as pd
from itertools import cycle
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs
from sklearn.cluster import SpectralClustering
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn import metrics

1. From Laplacian to Clustering

In [ ]:
# Computes the symetric normalized laplacian
# L = D^{-1/2} A D{-1/2}
def laplacian(A):
    D = numpy.zeros(A.shape)
    w = numpy.sum(A, axis=0)
    D.flat[::len(w) + 1] = w ** (-0.5)  # set the diag of D to w
    return D.dot(A).dot(D)

def k_means(X, n_clusters):
    kmeans = KMeans(n_clusters=n_clusters, random_state=1231)
    return kmeans.fit(X).labels_

def spectral_clustering(affinity, n_clusters, cluster_method=k_means):
    L = laplacian(affinity)
    eig_val, eig_vect = scipy.sparse.linalg.eigs(L, n_clusters)
    X = eig_vect.real
    rows_norm = numpy.linalg.norm(X, axis=1, ord=2)
    Y = (X.T / rows_norm).T
    labels = cluster_method(Y, n_clusters)
    return labels

2. Use sklearn Functions¶

In [ ]:
# Center for generating random data.
centers = [[1, 1], [-1, -1], [1, -1]]

# The number of data generated.
n_samples=3000

# Generate data.
X, lables_true = make_blobs(n_samples=n_samples, centers= centers, cluster_std=0.6, 
                  random_state =0)
 
# Transform to a matrix, the input has to be symmetric.
metrics_metrix = (-1 * metrics.pairwise.pairwise_distances(X)).astype(np.int32)
metrics_metrix += -1 * metrics_metrix.min()

# Set spectral clustering function.
n_clusters_= 4
lables = spectral_clustering(metrics_metrix,n_clusters=n_clusters_)

# Visualization of results
plt.figure(1)
plt.clf()
colors = cycle('bgrcmykbgrcmykbgrcmykbgrcmyk')
for k, col in zip(range(n_clusters_), colors):
    # Reassembles an array of True and False based on whether the values in lables are equal to k.
    my_members = lables == k
    plt.plot(X[my_members, 0], X[my_members, 1], col + '.')
    
plt.title('Estimated number of clusters: %d' % n_clusters_)
plt.show()